In [1]:
# # Dependencies
# import time
from splinter import Browser
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
from bs4 import BeautifulSoup
import pandas as pd
import re


In [2]:
all_data={}
url = 'https://mars.nasa.gov/news'

browser = webdriver.Chrome()  
browser.get(url)  

html_source = browser.page_source  
browser.quit()

soup = BeautifulSoup(html_source,'html.parser')  
mars_content = soup.find_all('li', class_="slide")


In [3]:
# len(mars_content)
# print(soup.prettify())

In [4]:
x=1
for result in mars_content:
    try:
        title = result.find('div', class_="content_title").text
        body = result.find('div', class_="article_teaser_body").text
     
        if (title and body):
            print(f'ARTICLE {x} ------------')
            print(title)
            print(body)
            x+=1
    except AttributeError as e:
        print(e)
        x+=1


ARTICLE 1 ------------
InSight Captures Sunrise and Sunset on Mars
InSight joins the rest of NASA's Red Planet surface missions, all of which have photographed either the start or end of a Martian day.
ARTICLE 2 ------------
NASA Social Media and Websites Win Webby Awards 
NASA's social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."
ARTICLE 3 ------------
NASA's InSight Detects First Likely 'Quake' on Mars
While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.
ARTICLE 4 ------------
Things Are Stacking up for NASA's Mars 2020 Spacecraft
As the July 2020 launch date inches closer, the next spacecraft headed to the Red Planet is assembled for more testing.
ARTICLE 5 ------------
Curiosity Tastes First Sample in 'Clay-Bearing Unit'
This new region on Mars might reveal more about the role o

In [5]:
all_data.update({'title':title, 'body':body})
all_data

{'title': 'NASA InSight Landing on Mars: Milestones',
 'body': "On Nov. 26, NASA's InSight spacecraft will blaze through the Martian atmosphere and set a lander gently on the surface in less time than it takes to cook a hard-boiled egg."}

In [6]:
####SECTION TWO: SPLINTER#####

In [7]:
!which chromedriver

/usr/local/bin/chromedriver


In [8]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url_2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_2)

html=browser.html
soup=BeautifulSoup(html, 'html.parser')

browser.find_by_id('full_image').first.click()
lightbox = soup.find_all('a', class_='button')

links=[]
for item in lightbox:
    url_item=item.get('data-link')
    if url_item != None:
        url_3='https://www.jpl.nasa.gov' + url_item
        browser.visit(url_3)
        browser.click_link_by_partial_href('.gov/jpeg/')
        featured_image_url=browser.url
        print(featured_image_url)
        browser.quit()
    else:
        continue

        

https://photojournal.jpl.nasa.gov/jpeg/PIA17200.jpg


In [9]:
all_data.update({'featured_image_url':featured_image_url})
all_data

{'title': 'NASA InSight Landing on Mars: Milestones',
 'body': "On Nov. 26, NASA's InSight spacecraft will blaze through the Martian atmosphere and set a lander gently on the surface in less time than it takes to cook a hard-boiled egg.",
 'featured_image_url': 'https://photojournal.jpl.nasa.gov/jpeg/PIA17200.jpg'}

In [10]:
##SECTION 3 Mars Weather###

In [11]:

url = 'https://twitter.com/marswxreport?lang=en'
tweets=[]
browser = webdriver.Chrome()  
browser.get(url)  

html_source = browser.page_source  
# browser.quit()

soup = BeautifulSoup(html_source,'html.parser')
mars_tweet=''
mars_tweets = soup.find_all('div', class_="js-tweet-text-container")





In [12]:
mars_weather_tweets=[]

for item in mars_tweets:
    if 'InSight sol' in item.text:
        mars_weather_tweets.append(item)
        if len(mars_weather_tweets)==1:
            break
    
latest_tweet=mars_weather_tweets[0].text
mars_weather=latest_tweet.replace('\n',' ') 
browser.quit()

In [13]:
mars_weather

' InSight sol 155 (2019-05-04) low -99.3ºC (-146.8ºF) high -18.8ºC (-1.8ºF) winds from the SW at 4.5 m/s (10.1 mph) gusting to 14.3 m/s (31.9 mph) pressure at 7.40 hPapic.twitter.com/wEcSHS2b3u '

In [14]:
all_data.update({'mars_weather':mars_weather})
all_data

{'title': 'NASA InSight Landing on Mars: Milestones',
 'body': "On Nov. 26, NASA's InSight spacecraft will blaze through the Martian atmosphere and set a lander gently on the surface in less time than it takes to cook a hard-boiled egg.",
 'featured_image_url': 'https://photojournal.jpl.nasa.gov/jpeg/PIA17200.jpg',
 'mars_weather': ' InSight sol 155 (2019-05-04) low -99.3ºC (-146.8ºF) high -18.8ºC (-1.8ºF) winds from the SW at 4.5 m/s (10.1 mph) gusting to 14.3 m/s (31.9 mph) pressure at 7.40 hPapic.twitter.com/wEcSHS2b3u '}

In [12]:
##SECTION 4 FACTS###

In [43]:

url_4='https://space-facts.com/mars/'

In [44]:
table = pd.read_html(url_4)
table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [47]:
df = table[0]
df.columns = ["title", "value"]
df.set_index("title", inplace=True)


In [68]:
table_titles=[]
table_values=[]
# x=0


for x in range(len(df)):
    table_titles.append(df.iloc[x].name[0:-1])
    table_values.append(df.iloc[x].value)
all_data['table_titles']=table_titles
all_data['table_values']=table_values
table_titles

['Equatorial Diameter',
 'Polar Diameter',
 'Mass',
 'Moons',
 'Orbit Distance',
 'Orbit Period',
 'Surface Temperature',
 'First Record',
 'Recorded By']

In [16]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>title</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </

In [47]:
all_data.update({'html_table':html_table})
all_data

{'title': 'NASA InSight Landing on Mars: Milestones',
 'body': "On Nov. 26, NASA's InSight spacecraft will blaze through the Martian atmosphere and set a lander gently on the surface in less time than it takes to cook a hard-boiled egg.",
 'featured_image_url': 'https://photojournal.jpl.nasa.gov/jpeg/PIA18904.jpg',
 'mars_weather': ' InSight sol 153 (2019-05-02) low -98.5ºC (-145.3ºF) high -17.5ºC (0.6ºF) winds from the SW at 4.7 m/s (10.5 mph) gusting to 11.9 m/s (26.6 mph) pressure at 7.40 hPapic.twitter.com/5RdVDek7mh ',
 'html_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>title</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</t

In [17]:
###SECTION 5 MARS HEMISPHERES ####

In [48]:
url_5='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [49]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(url_5)

html=browser.html
soup=BeautifulSoup(html, 'html.parser')



In [50]:
hemi_collection=[]
for item in soup.find_all('a', class_='itemLink product-item'):
    if item.text != '':
        hemi_collection.append(item)

In [54]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
# hemi_img=[]
hemi_img_links=[]
hemi_titles=[]
hemis_dicts_all=[]

for item in hemi_collection:
    link='https://astrogeology.usgs.gov' + item.get('href')
    browser.visit(link)
    html=browser.html
    soup=BeautifulSoup(html, 'html.parser')
#     hemi_titles.append(soup.find('h2',class_='title').text)
#     img_links=soup.find('a', attrs={'href': re.compile("^http://")})
#     img_link=img_links['href']
#     hemi_img_links.append(img_link)
    hemi_title=soup.find('h2',class_='title').text
    img_links=soup.find('a', attrs={'href': re.compile("^http://")})
    img_link=img_links['href']
    hemi_dict={}
    hemi_dict = {"title": hemi_title, "img_url":img_link}
    hemis_dicts_all.append(hemi_dict)
    x+=1    
browser.quit()

In [57]:
hemis_dicts_all

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [23]:
# dicts=[]
# dict_={}
# x=0
# while (x<4):
#     dict_["title"]=(hemi_titles[x])
#     dict_["img_url"]=(hemi_img_links[x])
#     dicts.append(dict_)
#     x+=1
# dicts

In [58]:
all_data.update({'hemis_dicts_all':hemis_dicts_all})
all_data

{'title': 'NASA InSight Landing on Mars: Milestones',
 'body': "On Nov. 26, NASA's InSight spacecraft will blaze through the Martian atmosphere and set a lander gently on the surface in less time than it takes to cook a hard-boiled egg.",
 'featured_image_url': 'https://photojournal.jpl.nasa.gov/jpeg/PIA18904.jpg',
 'mars_weather': ' InSight sol 153 (2019-05-02) low -98.5ºC (-145.3ºF) high -17.5ºC (0.6ºF) winds from the SW at 4.7 m/s (10.5 mph) gusting to 11.9 m/s (26.6 mph) pressure at 7.40 hPapic.twitter.com/5RdVDek7mh ',
 'html_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>title</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</t